In [1]:
import tensorflow as tf
import numpy as np

from math import exp

In [2]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz


Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# Learning Rate Values
lrmax = 0.003
lrmin = 0.00001
decay_speed = 2000.0
stddev = 0.1

In [14]:
# Place Holders for loading data on training
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y_ = tf.placeholder(tf.float32, [None, 10])
L = tf.placeholder(tf.float32)

In [5]:
# Filter is the block of data (pixels) to scan an image and produce a value
filter_size = [5, 5]

# A grey scall image would have one, RGB would have 3
input_channel = 1

# Number of channels to produce. Each layer serves as a shape recognisers.
output_channel = 4

In [6]:
image = [
    [0.11, 0.12, 0.13, 0.14],
    [0.21, 0.22, 0.23, 0.24],
    [0.31, 0.32, 0.33, 0.34],
    [0.50, 0.60, 0.70, 0.80]
]


In [7]:
# Weights

# Convolutional Layers
# 5x5x1x4
w_shape1 = filter_size + [input_channel] + [output_channel]
W1 = tf.Variable(tf.truncated_normal(w_shape1, stddev=stddev))
b_shape1 = [output_channel]
B1 = tf.Variable(tf.ones(b_shape1)/10)

# L2
W2 = tf.Variable(tf.truncated_normal([4, 4, 4, 8], stddev=stddev))
B2 = tf.Variable(tf.ones([8])/10)

# L3
W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=stddev))
B3 = tf.Variable(tf.ones([12])/10)

# Fully Connected Layer
W4 = tf.Variable(tf.truncated_normal([7*7*12, 200], stddev=stddev))
B4 = tf.Variable(tf.ones([200])/10)

# Softmax Readout Layer
W5 = tf.Variable(tf.truncated_normal([200, 10], stddev=stddev))
B5 = tf.Variable(tf.ones([10])/10)

In [8]:
# The model
stride = 1  # output is 28x28
Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
stride = 2  # output is 14x14
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
stride = 2  # output is 7x7
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y3, shape=[-1, 7 * 7 * 12])

Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [9]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
train_step = tf.train.AdamOptimizer(L).minimize(cross_entropy)

In [10]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [11]:
batch_X, batch_Y = mnist.train.next_batch(100)
learning_rate = lrmin + (lrmax - lrmin) * exp(-1/decay_speed)
sess.run(train_step, {X: batch_X, Y_: batch_Y, L: learning_rate})

In [12]:
t = sess.run(Y1, feed_dict={X:batch_X})

In [17]:
t.shape

(100, 28, 28, 4)